# **Topic Modelling**

Tutorial: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

Explicación: https://medium.com/@soorajsubrahmannian/extracting-hidden-topics-in-a-corpus-55b2214fc17d

Explicación 2: https://slides.cpsievert.me/ldavis/#9

Paper: https://www.aclweb.org/anthology/W14-3110


In [0]:
# Imports

from google.colab import drive
drive.mount('/content/gdrive') # grant access to google drive filesystem

!python3 -m spacy download es
!pip install pyldavis
import nltk; nltk.download('stopwords')


import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

from nltk.corpus import stopwords

import os
from datetime import datetime

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
     |████████████████████████████████| 36.7MB 1.8MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-tr3c29n5/wheels/9e/28/c4/df4980946eb229379ed26d349566e427fa029dbf03546ccb94
Successfully built es-core-news-sm

    Linking successful
    /usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/es

    You can now load the model via spacy.load('es')

     |████████████████████████████████| 1.6MB 4.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# Initializations

notesFile = 'content.csv'
sitio = 'Expansión'
path = '/content/gdrive/My Drive/datos/'
start_date = pd.to_datetime('01-01-2019') # mes-dia-año
end_date = pd.to_datetime('8-01-2019')

stop_words = stopwords.words('spanish')
stop_words.extend(['desde', 'expansión', 'publicidad', 'adnpolítico', 'getty images', 'con', '¿'])

# Stop words, palabras a ignorar
stop_words = STOP_WORDS
stop_words.add('expansión')
stop_words.add('publicidad')
stop_words.add('adnpolítico')
stop_words.add('adnpolítico)')
stop_words.add('adnpolítico).-')
stop_words.add('getty images')
stop_words.add('méxico')
stop_words.add('con)')
stop_words.add('“')
stop_words.add('\n')
stop_words.add('¿')

# Stop lines, líneas a ignorar
stop_lines = set()
stop_lines.add('te puede interesar:')
stop_lines.add('Recomendamos:')
stop_lines.add('También Lee:')
stop_lines.add('Con información de Notimex')
stop_lines.add('Notimex')
stop_lines.add('Publicidad')
stop_lines.add('\n')

In [0]:
# Import Dataset
with open(os.path.join(path, notesFile)) as csv_file:
    df = pd.read_csv(csv_file)
    
# Convert fecha to datetime
df['datetime'] = pd.to_datetime(df['fecha'])
#df = df.set_index('datetime')
#df.drop(['fecha'], axis=1, inplace=True)

# Filtrar por sitio <--- FILTROS!!!!! #############################################################################################
df = df.loc[df['sitio'] == sitio]

# Filtrar por fecha
df = df.loc[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)]

# Filtrar por pageviews
df = df.loc[(df['pageviews'] > df.loc[:,"pageviews"].median() * 1.1 ) ]

# Filtrar por seccion
df = df.loc[(df['seccion'] == 'empresas' ) ]


#print(df.texto.unique())
df.head()

,fecha,sitio,url,seccion,autor,título,sumario,pageviews,texto,datetime
48,2019-01-02 00:00:00,Expansión,https://expansion.mx/empresas/2019/01/02/el-cc...,empresas,Notimex,El CCE exhorta a impulsar mayor crecimiento de...,Los empresarios piden impulsar crecimiento eco...,764,CIUDAD DE MÉXICO - Los empresarios del país se...,2019-01-02
49,2019-01-02 00:00:00,Expansión,https://expansion.mx/empresas/2019/01/02/las-g...,empresas,Expansión,Las ganadoras y perdedoras de la BMV en 2018,De las 35 emisoras que conforman el principal ...,1314,El 2018 quedó atrás y con él uno de los peores...,2019-01-02
50,2019-01-02 00:00:00,Expansión,https://expansion.mx/empresas/2019/01/02/un-mo...,empresas,Reuters,Un Model 3 más barato y menos entregas hacen c...,Las acciones de Tesla caen tras reporte de Mod...,1412,Las acciones de Tesla Inc llegaron a caer un 9...,2019-01-02
51,2019-01-02 00:00:00,Expansión,https://expansion.mx/empresas/2019/01/02/borro...,empresas,Ana Valle,"El borrón y cuenta nueva de Aleatica, el nuevo...",OHL México ya no existe. Después de varios esc...,2252,"En 2015, los directivos de OHL México dejaron ...",2019-01-02
52,2019-01-02 00:00:00,Expansión,https://expansion.mx/empresas/2019/01/02/joyer...,empresas,Liliana Corona,Joyería de Taxco para el mundo,Joyería de Taxco para el mundo,2820,Taxco es una pequeña ciudad del sureño estado ...,2019-01-02


In [0]:
def delete_stop_lines(text):
# cleans stop lines    
    article = ''
    for line in str(text).splitlines():
        borrar = False
        for sl in stop_lines:
            if sl in line:
                borrar = True
        if not borrar and len(line) > 5:
            article = article + line
    return(article)

In [0]:
# Limpiar datos

data = df.texto.values.tolist()

#data = [x.strip().lower() for x in data]
#data = [x for x in data if x not in stop_words]
#data = [x for x in data if x != '']

# delete stop lines
data = [delete_stop_lines(text) for text in data]

# Remove Emails
data = [re.sub('\*@\S*\s?', '', str(sent)) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

data = [delete_stop_lines(text) for text in data]

pprint(data[:1])

['CIUDAD DE MÉXICO - Los empresarios del país se pronunciaron por acelerar la '
 'marcha de la economía en el país, a través del impulso a la competitividad, '
 'apertura de mercados internacionales y estabilidad económica para superar el '
 'crecimiento de 2 y 2.5% del Producto Interno Bruto (PIB).El Consejo '
 'Coordinador Empresarial (CCE) calificó como positivo el balance de 2018 para '
 'México, aunque reconoció que éste es aún insuficiente en términos '
 'económicos, políticos y sociales.“Con el trabajo decido de todos los '
 'mexicanos, logramos superar retos importantes durante el año, pero aún no '
 'alcanzamos las tasas de crecimiento necesarias para terminar con la pobreza '
 'y cerrar las brechas de desigualdad”, expuso la cúpula empresarial.Lee: El '
 'envío de remesas a México en 2018 alcanzará récord históricoEn su documento '
 'semanal La Voz del CCE, el organismo destacó que la conclusión de las '
 'negociaciones del Tratado entre México, Estados Unidos y Canadá, permi

In [0]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True, min_len=2, max_len=15))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['ciudad', 'de', 'mexico', 'los', 'empresarios', 'del', 'pais', 'se', 'pronunciaron', 'por', 'acelerar', 'la', 'marcha', 'de', 'la', 'economia', 'en', 'el', 'pais', 'traves', 'del', 'impulso', 'la', 'competitividad', 'apertura', 'de', 'mercados', 'internacionales', 'estabilidad', 'economica', 'para', 'superar', 'el', 'crecimiento', 'de', 'del', 'producto', 'interno', 'bruto', 'pib', 'el', 'consejo', 'coordinador', 'empresarial', 'cce', 'califico', 'como', 'positivo', 'el', 'balance', 'de', 'para', 'mexico', 'aunque', 'reconocio', 'que', 'este', 'es', 'aun', 'insuficiente', 'en', 'terminos', 'economicos', 'politicos', 'sociales', 'con', 'el', 'trabajo', 'decido', 'de', 'todos', 'los', 'mexicanos', 'logramos', 'superar', 'retos', 'importantes', 'durante', 'el', 'ano', 'pero', 'aun', 'no', 'alcanzamos', 'las', 'tasas', 'de', 'crecimiento', 'necesarias', 'para', 'terminar', 'con', 'la', 'pobreza', 'cerrar', 'las', 'brechas', 'de', 'desigualdad', 'expuso', 'la', 'cupula', 'empresarial', 'l

In [0]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['ciudad', 'de', 'mexico', 'los', 'empresarios', 'del', 'pais', 'se', 'pronunciaron', 'por', 'acelerar', 'la', 'marcha', 'de', 'la', 'economia', 'en', 'el', 'pais', 'traves', 'del', 'impulso', 'la', 'competitividad', 'apertura', 'de', 'mercados', 'internacionales', 'estabilidad', 'economica', 'para', 'superar', 'el', 'crecimiento', 'de', 'del', 'producto', 'interno', 'bruto', 'pib', 'el', 'consejo_coordinador_empresarial_cce', 'califico', 'como', 'positivo', 'el', 'balance', 'de', 'para', 'mexico', 'aunque', 'reconocio', 'que', 'este', 'es', 'aun', 'insuficiente', 'en', 'terminos', 'economicos', 'politicos', 'sociales', 'con', 'el', 'trabajo', 'decido', 'de', 'todos', 'los', 'mexicanos', 'logramos', 'superar', 'retos', 'importantes', 'durante', 'el', 'ano', 'pero', 'aun', 'no', 'alcanzamos', 'las', 'tasas', 'de', 'crecimiento', 'necesarias', 'para', 'terminar', 'con', 'la', 'pobreza', 'cerrar', 'las', 'brechas', 'de', 'desigualdad', 'expuso', 'la', 'cupula', 'empresarial', 'lee', 'el',

In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        #texts_out.append([token.text for token in doc if token.pos_ in allowed_postags]) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags]) 
    return texts_out

In [0]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'es' model, keeping only tagger component (for efficiency)
# python3 -m spacy download es
nlp = spacy.load('es', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['ciudad', 'mexico', 'empresario', 'pronunciar', 'acelerar', 'marchar', 'economia', 'traves', 'impulsar', 'competitividad', 'apertura', 'mercar', 'internacional', 'estabilidad', 'economica', 'superar', 'producto', 'internar', 'bruto', 'consejo_coordinador', 'empresarial_cce', 'calificar', 'positivo', 'balance', 'mexico', 'reconocio', 'insuficiente', 'terminos', 'economicos', 'politicos', 'social', 'decidir', 'mexicano', 'lograr', 'superar', 'reto', 'importante', 'alcanzar', 'tasar', 'crecimiento', 'necesario', 'terminar', 'pobreza', 'cerrar', 'brecha', 'desigualdad', 'exponer', 'cupula', 'empresarial', 'leer', 'envio', 'remesar', 'mexico', 'alcanzar', 'record', 'historicoen', 'documentar', 'semanal', 'voz', 'organismo', 'destacar', 'conclusion', 'negociación', 'tratar', 'mexico', 'unir', 'canada', 'permitira', 'consolidar', 'potenciar', 'exportador', 'forjar', 'region', 'competitivo', 'nivel_global', 'refirio', 'lograr', 'marcar', 'juridico', 'mejorar', 'regulatoria', 'politica', 'sim

In [0]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

# Human readable format of corpus (term-frequency)
# pprint([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, # The number of requested latent topics to be extracted from the training corpus.
                                           random_state=100, # Either a randomState object or a seed to generate one. Useful for reproducibility
                                           update_every=1, # Number of documents to be iterated through for each update. Set to 0 for batch learning, > 1 for online iterative learning
                                           chunksize=100, # Number of documents to be used in each training chunk.
                                           passes=10, # Number of passes through the corpus during training.
                                           alpha='auto', # ’auto’: Learns an asymmetric prior from the corpus 
                                           per_word_topics=True)

In [0]:
# Print the Keyword in the n topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.039*"aeropuerto" + 0.026*"aeronave" + 0.022*"aerolineas" + 0.018*"avión" '
  '+ 0.013*"cfe" + 0.010*"generacion" + 0.009*"aicm" + 0.009*"gasoducto" + '
  '0.009*"mecanismo" + 0.009*"aeromexico"'),
 (1,
  '0.041*"tienda" + 0.030*"marcar" + 0.023*"walmart" + 0.012*"minorista" + '
  '0.012*"producto" + 0.012*"venta" + 0.011*"tender" + 0.011*"soriano" + '
  '0.010*"cadena" + 0.009*"subastar"'),
 (2,
  '0.024*"presidente" + 0.021*"consejo" + 0.020*"empresario" + '
  '0.018*"trabajador" + 0.015*"empresa" + 0.013*"empresarial" + 0.013*"holgar" '
  '+ 0.011*"promocion" + 0.011*"sindicato" + 0.010*"promexico"'),
 (3,
  '0.029*"vivienda" + 0.013*"sector" + 0.011*"venta" + 0.009*"infonavit" + '
  '0.009*"precio" + 0.009*"cemex" + 0.008*"centro" + 0.007*"costo" + '
  '0.007*"concretar" + 0.007*"exclusivo"'),
 (4,
  '0.026*"ano" + 0.026*"millón" + 0.018*"crecimiento" + 0.016*"venta" + '
  '0.014*"ingreso" + 0.013*"empresa" + 0.012*"trimestre" + 0.011*"incrementar" '
  '+ 0.011*"peso" + 0.

In [0]:
lda_model.num_terms

15167

In [0]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.247916919100247

Coherence Score:  0.41315451890386923


In [0]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.258526 -0.040705       1        1  27.381302
16    -0.239329  0.048709       2        1  17.475306
4     -0.261928  0.027927       3        1   7.941959
6     -0.172947 -0.183152       4        1   7.206064
7     -0.149262  0.003832       5        1   5.264284
17    -0.164218  0.135648       6        1   4.713713
1      0.024069  0.205685       7        1   3.499946
19     0.003455  0.089586       8        1   3.285511
13     0.029093 -0.197301       9        1   2.961460
3     -0.030158 -0.023306      10        1   2.891212
14     0.102247  0.130727      11        1   2.763626
2      0.048379 -0.126563      12        1   2.505840
12     0.136745  0.047932      13        1   1.865524
11     0.121847  0.000314      14        1   1.705875
8      0.092800 -0.015260      15        1   1.685763
5      0.147952  0.058624      16        1   1.673757
0      0.130515 -0.113433      17        1   1.632164
15     0.124934 -0.024480      18        1   1.486462
9      0.154394 -0.023896      19        1   1.305080
18     0.159939 -0.000891      20        1   0.755154, topic_info=      Category         Freq  ...  loglift  logprob
112    Default  1812.000000  ...  30.0000  30.0000
2103   Default   651.000000  ...  29.0000  29.0000
773    Default   451.000000  ...  28.0000  28.0000
54     Default  1603.000000  ...  27.0000  27.0000
688    Default  1243.000000  ...  26.0000  26.0000
337    Default   261.000000  ...  25.0000  25.0000
801    Default   532.000000  ...  24.0000  24.0000
105    Default   455.000000  ...  23.0000  23.0000
5      Default  1263.000000  ...  22.0000  22.0000
80     Default   683.000000  ...  21.0000  21.0000
386    Default   230.000000  ...  20.0000  20.0000
460    Default   364.000000  ...  19.0000  19.0000
110    Default   686.000000  ...  18.0000  18.0000
101    Default  1037.000000  ...  17.0000  17.0000
2105   Default   273.000000  ...  16.0000  16.0000
245    Default   634.000000  ...  15.0000  15.0000
139    Default   422.000000  ...  14.0000  14.0000
178    Default   506.000000  ...  13.0000  13.0000
549    Default   808.000000  ...  12.0000  12.0000
496    Default   158.000000  ...  11.0000  11.0000
408    Default   255.000000  ...  10.0000  10.0000
159    Default   381.000000  ...   9.0000   9.0000
22     Default   236.000000  ...   8.0000   8.0000
2036   Default   216.000000  ...   7.0000   7.0000
477    Default   161.000000  ...   6.0000   6.0000
351    Default   300.000000  ...   5.0000   5.0000
2062   Default   207.000000  ...   4.0000   4.0000
23     Default   607.000000  ...   3.0000   3.0000
37     Default   308.000000  ...   2.0000   2.0000
1704   Default   326.000000  ...   1.0000   1.0000
...        ...          ...  ...      ...      ...
7122   Topic20    11.540879  ...   4.8192  -4.6528
14601  Topic20    11.403156  ...   4.8184  -4.6648
3634   Topic20    11.364857  ...   4.8182  -4.6681
6817   Topic20     8.158037  ...   4.7928  -4.9997
3476   Topic20     8.155425  ...   4.7927  -5.0000
3437   Topic20     8.117042  ...   4.7923  -5.0047
5118   Topic20     7.086207  ...   4.7794  -5.1405
8520   Topic20     6.961126  ...   4.7776  -5.1583
5136   Topic20     6.907072  ...   4.7768  -5.1661
1069   Topic20     6.560256  ...   4.7713  -5.2176
8016   Topic20     6.260805  ...   4.7662  -5.2644
14600  Topic20     5.717672  ...   4.7555  -5.3551
1119   Topic20     5.677139  ...   4.7546  -5.3622
12135  Topic20     5.582893  ...   4.7525  -5.3790
10062  Topic20     5.483403  ...   4.7503  -5.3969
1143   Topic20     5.165535  ...   4.7425  -5.4567
9926   Topic20     5.127383  ...   4.7415  -5.4641
1162   Topic20     4.972888  ...   4.7373  -5.4947
10063  Topic20     4.961644  ...   4.7370  -5.4969
12901  Topic20     4.673027  ...   4.7285  -5.5569
6602   Topic20     4.665483  ...   4.7283  -5.5585
10057  Topic20     4.632885  ...   4.7272  -5.5655
6819   Topic20 

In [0]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

NameError: ignored